In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from cryptocmd import CmcScraper #market cap data
from rich.progress import track
from IPython.display import clear_output

In [2]:
#Import all csv and save them in dict with filename (without extension, no USDT) as key
path = "Crypto_Data_Hourly//"
file_names = [f for f in listdir(path) if isfile(join(path, f))]
file_names.remove("BTTCUSDT.csv") #super tiny & no market cap data

dataframe_dict = {}

for file_name in track(file_names):
    dataframe_dict[file_name[:-8]] = pd.read_csv(path + file_name)
    dataframe_dict[file_name[:-8]]["Open Time"] = pd.to_datetime(dataframe_dict[file_name[:-8]]["Open Time"])

clear_output()
dataframe_dict.keys()

dict_keys(['1INCH', 'AAVE', 'ADA', 'ALGO', 'AMP', 'APE', 'AR', 'ATOM', 'AVAX', 'AXS', 'BAL', 'BAT', 'BCH', 'BNB', 'BNX', 'BTC', 'BTG', 'CAKE', 'CELO', 'COMP', 'CVX', 'DASH', 'DCR', 'DOGE', 'DOT', 'EGLD', 'ENJ', 'ENS', 'EOS', 'ETC', 'ETH', 'FIL', 'FLOW', 'FTM', 'FTT', 'GALA', 'GLMR', 'GMT', 'GRT', 'HBAR', 'HIVE', 'HNT', 'HOT', 'ICP', 'ICX', 'IMX', 'IOST', 'IOTA', 'IOTX', 'JST', 'KAVA', 'KDA', 'KLAY', 'KNC', 'KSM', 'LDO', 'LINK', 'LPT', 'LRC', 'LTC', 'MANA', 'MATIC', 'MINA', 'MKR', 'NEAR', 'NEO', 'NEXO', 'OMG', 'ONE', 'ONT', 'PAXG', 'QNT', 'QTUM', 'ROSE', 'RUNE', 'RVN', 'SAND', 'SC', 'SHIB', 'SNX', 'SOL', 'SRM', 'STORJ', 'STX', 'THETA', 'TRX', 'T', 'TUSD', 'TWT', 'UNI', 'USDP', 'VET', 'WAXP', 'WOO', 'XEC', 'XEM', 'XLM', 'XMR', 'XRP', 'XTZ', 'YFI', 'ZEC', 'ZEN', 'ZIL', 'ZRX'])

In [3]:
#Some datacleaning
#Each dataframe has Open Time twice for some reason. Let's drop that column from all df in the dict
#Let's also add return as pct_change of closing price
for crypto in dataframe_dict:
    dataframe_dict[crypto].drop('Open Time.1', inplace = True, axis = 1)
    dataframe_dict[crypto]["Return"] = dataframe_dict["BTC"]["Close"].pct_change()

In [4]:
print("See example dict entry for BTC:")
dataframe_dict["BTC"]

See example dict entry for BTC:


,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore,Return
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,2017-08-17 23:59:59.999000064,3.454770e+06,3427,616.248541,2.678216e+06,8733.911395,NaN
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,2017-08-18 23:59:59.999000064,5.086958e+06,5233,972.868710,4.129123e+06,9384.141409,-0.041238
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,2017-08-19 23:59:59.999000064,1.549484e+06,2153,274.336042,1.118002e+06,9184.085529,0.007694
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,2017-08-20 23:59:59.999000064,1.930364e+06,2321,376.795947,1.557401e+06,10125.414084,-0.012969
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2017-08-21 23:59:59.999000064,2.797232e+06,3972,557.356107,2.255663e+06,11706.769970,-0.017201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2022-07-17,21195.60,21684.54,20750.01,20798.16,118229.452500,2022-07-17 23:59:59.999000064,2.511251e+09,4261057,59550.952910,1.265043e+09,0.000000,-0.018751
1796,2022-07-18,20799.58,22777.63,20762.45,22432.58,239942.731320,2022-07-18 23:59:59.999000064,5.263397e+09,6564387,122398.318620,2.685273e+09,0.000000,0.078585
1797,2022-07-19,22432.58,23800.00,21579.54,23396.62,263770.765740,2022-07-19 23:59:59.999000064,5.954420e+09,6488375,132874.434050,2.999606e+09,0.000000,0.042975
1798,2022-07-20,23398.48,24276.74,22906.19,23223.30,238762.170940,2022-07-20 23:59:59.999000064,5.631801e+09,6364038,119820.553520,2.826789e+09,0.000000,-0.007408


In [5]:
#Let's identify stable coins by their low variance
variances = {}

#variances_df["crypto"] = dataframe_dict.keys()

for crypto in dataframe_dict:
    variances[crypto] = [dataframe_dict[crypto]['Open'].var()]

variances_df = pd.DataFrame(data = variances).T
variances_df = variances_df.rename(columns={0: "Variance"})
#display(variances_df.sort_values("Variance").head(10))

list_of_stable_coins = ["USDP", "TUSD"]
list_of_stable_coins

['USDP', 'TUSD']

In [6]:
#getting marketcap data
## fixing some names for coinmarketcap data and deleting previous Market Cap columns if exist
coinmarketcap_names = {}
for crypto in dataframe_dict:
    coinmarketcap_names[crypto] = crypto
    dataframe_dict[crypto].drop(list(dataframe_dict[crypto].filter(regex = 'Market Cap')), axis = 1, inplace = True)

coinmarketcap_names["IOTA"] = "MIOTA" #actually only iota had problems

#start and end date for faster execution. Format given by CmcScraper library
start = dataframe_dict["BTC"]["Open Time"].iloc[0].strftime("%d-%m-%Y")
end = dataframe_dict["BTC"]["Open Time"].iloc[-1].strftime("%d-%m-%Y")

##scraping coinmarketcap and joining with original dfs
for crypto in track(dataframe_dict):
    scraper = CmcScraper(coinmarketcap_names[crypto], start, end)
    market_cap = scraper.get_dataframe()[["Date", "Market Cap"]]
    market_cap = market_cap.rename(columns = {"Date": "Open Time", "Market Cap": "Market Cap"})
    dataframe_dict[crypto] = pd.merge(dataframe_dict[crypto], market_cap, on = "Open Time")

clear_output()

In [7]:
#market cap dataframe: Building a df with coins as columns and open time as rows. Entries are the market cap of each coin for given time.
all_market_caps = pd.DataFrame()
all_market_caps["Open Time"] = dataframe_dict["BTC"]["Open Time"] #Bitcoin because it has the largest range of dates

for crypto in (dataframe_dict):
    all_market_caps = pd.merge(all_market_caps, dataframe_dict[crypto][["Open Time", "Market Cap"]], on = "Open Time", how = "outer")
    all_market_caps = all_market_caps.rename(columns={all_market_caps.columns[-1]: crypto})
all_market_caps = all_market_caps.fillna(0)

print("Beginning of dataframe for BTC, ETH & 1Inch. 1Inch is 0 because it didn't exist back then")
display(all_market_caps[["Open Time", "BTC", "ETH", "1INCH"]].head(3))

print("That's the whole dataframe. Lots of coins didn't exist in 2017")
all_market_caps.head(3)

Beginning of dataframe for BTC, ETH & 1Inch. 1Inch is 0 because it didn't exist back then


,Open Time,BTC,ETH,1INCH
0,2017-08-17,7.152795e+10,2.835757e+10,0.0
1,2017-08-18,6.871087e+10,2.781228e+10,0.0
2,2017-08-19,6.926514e+10,2.799589e+10,0.0


That's the whole dataframe. Lots of coins didn't exist in 2017


,Open Time,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,AVAX,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
0,2017-08-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-08-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-08-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
#relative market cap dataframe: Basically transforming the dataframe from above to percentage so that each row sums up to 100%
rel_market_caps = all_market_caps.copy(deep=True)

#drop stable coins
print(f"Setting market cap of stable coins to 0: {list_of_stable_coins}")
rel_market_caps[list_of_stable_coins] = 0
#rel_market_caps.drop(list_of_stable_coins, inplace=True, axis = 1)

#transpose dataframe
rel_market_caps = rel_market_caps.transpose()
new_header = rel_market_caps.iloc[0] #grab the first row for the header
rel_market_caps = rel_market_caps[1:] #take the data less the header row
rel_market_caps.columns = new_header #set the header row as the df header

#divide each column by sum of market caps to get relative market cap
rel_market_caps = rel_market_caps / rel_market_caps.sum()
assert sum(rel_market_caps.sum()) == 1800 #assert if sums of all dates are 1 * 1800 dates

#transpose dataframe back
rel_market_caps = rel_market_caps.transpose()
rel_market_caps = rel_market_caps.reset_index(level=0)
assert rel_market_caps.columns.all() == all_market_caps.columns.all() #assert if transposing was successful
assert sum(rel_market_caps.loc[:, rel_market_caps.columns != "Open Time"].sum(axis = 1)) == 1800 #assert if sums of all dates are 1 * 1800 dates

rel_market_caps.tail(3)

Setting market cap of stable coins to 0: ['USDP', 'TUSD']


,Open Time,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,AVAX,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
1797,2022-07-19,0.000478,0.001541,0.020271,0.002992,0.000463,0.002161,0.0006,0.003498,0.008133,...,0.000545,0.003452,0.00309,0.020898,0.00182,0.000286,0.001115,0.000264,0.000665,0.000326
1798,2022-07-20,0.000449,0.001492,0.019567,0.002828,0.000469,0.002128,0.000546,0.00328,0.007908,...,0.000512,0.00332,0.00325,0.020574,0.001726,0.000281,0.001091,0.000249,0.000632,0.000314
1799,2022-07-21,0.000451,0.001548,0.019655,0.002826,0.000454,0.002292,0.000555,0.003692,0.008279,...,0.000514,0.003323,0.003192,0.020662,0.001762,0.000287,0.001073,0.00025,0.000633,0.000318


In [10]:
avg_return_df = pd.DataFrame()
avg_return_df["Open Time"] = dataframe_dict["BTC"]["Open Time"] #Bitcoin because it has the largest range of dates
avg_return_df[["Average"]] = 0

for crypto in dataframe_dict:
    #get data on same dates by performing outer merge of dataframes on open time
    avg_return_df = pd.merge(avg_return_df, dataframe_dict[crypto][["Open Time", "Return"]], on = "Open Time", how = "outer", suffixes = (None, "_y"))
    avg_return_df = avg_return_df.rename(columns={avg_return_df.columns[-1]: crypto})

    values = avg_return_df[crypto]
    weights = rel_market_caps[crypto]
    assert len(values) == len(weights), f"{crypto} len of weights and values mismatch"
    avg_return = (values * weights).fillna(0)

    avg_return_df["Average"] += avg_return
    #avg_return_df.drop([crypto], inplace=True, axis = 1)

    assert avg_return_df["Average"].isnull().sum().any() == 0, f"{crypto}"

display(avg_return_df.head(3))
display(avg_return_df.tail(3))
display(avg_return_df.describe())

,Open Time,Average,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
0,2017-08-17,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-18,-0.041238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-19,0.007694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Open Time,Average,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
1797,2022-07-19,0.028892,-0.011543,-0.004192,-0.057225,-0.010353,0.139379,-0.082787,-0.001352,-0.000648,...,-0.054798,0.045649,-0.031370,-0.006341,-0.061163,-0.005655,0.007954,0.009065,0.047446,0.061297
1798,2022-07-20,-0.004543,0.002872,-0.034971,0.020401,0.032802,-0.005754,-0.046615,-0.004207,-0.048885,...,0.006107,0.074662,0.048024,0.037906,0.019905,-0.035371,0.098273,0.000306,-0.061370,0.083059
1799,2022-07-21,-0.018652,-0.004309,0.029243,0.028513,0.010639,0.015437,-0.060425,0.001830,0.044324,...,-0.000553,-0.027656,-0.024276,-0.005117,0.000867,0.006861,0.068507,0.014343,-0.020337,0.072422


,Average,1INCH,AAVE,ADA,ALGO,AMP,APE,AR,ATOM,AVAX,...,XEM,XLM,XMR,XRP,XTZ,YFI,ZEC,ZEN,ZIL,ZRX
count,1800.000000,573.000000,644.000000,1556.000000,1125.000000,240.000000,126.000000,433.000000,1179.000000,667.000000,...,604.000000,1512.000000,1224.000000,1539.000000,1031.000000,710.000000,1218.000000,745.000000,1248.000000,1239.000000
mean,0.002077,0.000873,0.001950,0.002612,0.001777,0.004371,0.011707,0.002175,0.002016,0.002074,...,0.001297,0.002606,0.002297,0.002670,0.001785,0.002163,0.002291,0.002112,0.002629,0.002711
std,0.035028,0.045840,0.044967,0.042485,0.043204,0.059492,0.054600,0.048784,0.042543,0.044611,...,0.045329,0.042790,0.042306,0.042594,0.044517,0.045491,0.042360,0.044983,0.042489,0.042392
min,-0.318712,-0.194974,-0.194974,-0.395048,-0.395048,-0.194974,-0.191566,-0.194974,-0.395048,-0.194974,...,-0.194974,-0.395048,-0.395048,-0.395048,-0.395048,-0.194974,-0.395048,-0.194974,-0.395048,-0.395048
25%,-0.013582,-0.018088,-0.015887,-0.015769,-0.015541,-0.025576,-0.014705,-0.018874,-0.015141,-0.015589,...,-0.016523,-0.015820,-0.015066,-0.015726,-0.016422,-0.016008,-0.014852,-0.016017,-0.015066,-0.014796
50%,0.001507,0.001478,0.002108,0.001834,0.001580,0.004734,0.008511,0.002489,0.001793,0.002234,...,0.001754,0.001834,0.002039,0.001839,0.001340,0.002339,0.002039,0.002321,0.002098,0.002168
75%,0.016850,0.018319,0.018762,0.020491,0.018347,0.036096,0.038478,0.023469,0.017942,0.018863,...,0.017959,0.020401,0.018486,0.020540,0.019184,0.019873,0.018411,0.019925,0.019123,0.019011
max,0.223402,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,...,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014,0.225014


In [55]:
#Save return dataframe in csv
avg_return_df.to_csv("daily_return.csv", index = False)

In [28]:
# import matplotlib.pyplot as plt

# fig1, ax1 = plt.subplots()
# ax1.set_title('Basic Plot')
# ax1.boxplot(avg_return_df["Average"], showfliers=False)

In [31]:
sum_columns = ["Volume", "Number of Trades"]
sum_columns + ["Open Time"]

['Volume', 'Number of Trades', 'Open Time']

In [86]:
market_coin_df = pd.DataFrame()
market_coin_df["Open Time"] = dataframe_dict["BTC"]["Open Time"] #Bitcoin because it has the largest range of dates
market_coin_df["Close"] = 100
market_coin_df["Return"] = avg_return_df["Average"]

sum_columns = ["Volume", "Number of Trades"]
market_coin_df[sum_columns] = 0

for crypto in dataframe_dict:

    market_coin_df = pd.merge(market_coin_df, dataframe_dict[crypto][sum_columns + ["Open Time"]], on = "Open Time", how = "outer", suffixes = (None, "_y"))
    
    for column in sum_columns:
        market_coin_df[column+"_y"] = market_coin_df[column+"_y"].fillna(0)
        market_coin_df[column] += market_coin_df[column+"_y"]
        market_coin_df.drop([column+"_y"], inplace=True, axis = 1)

#assert if sum works:
for column in sum_columns:
    test_sum = 0
    sum_market_coin = market_coin_df[column].sum()
    for crypto in dataframe_dict:
        test_sum += dataframe_dict[crypto][column].sum()
    assert round(sum_market_coin, -2) == round(test_sum, -2)


#save / load
#market_coin_df.to_csv("market_coin_df.csv", index = False, sep=';')
market_coin_df = pd.read_csv("market_coin_df.csv", delimiter=';') #index = False)
market_coin_df.head(3)

,Open Time,Close,Return,Volume,Number of Trades
0,17.08.2017,100.000000,0.000000,7825.860717,7949
1,18.08.2017,95.876156,-0.041238,10737.734720,10891
2,19.08.2017,96.613832,0.007694,2527.507493,3948
